In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend
from sklearn.metrics import classification_report, confusion_matrix

Checking backend

In [120]:
backend.backend()

'tensorflow'

setting random seed

In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
data_features = pd.DataFrame()
for i in range(10):
    filename = "/content/drive/My Drive/safety/features/part-0000"+str(i)+"-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv"
    temp_data_features = pd.read_csv(filename, header=0)
    data_features = data_features.append(temp_data_features)

In [0]:
data_features.columns = ['bookingID', 'accuracy', 'bearing', 
                    'acceleration_x', 'acceleration_y', 'acceleration_z',
                    'gyro_x', 'gyro_y', 'gyro_z',
                    'second','speed']

data_features = data_features.loc[(data_features["accuracy"]<20) &
                (data_features["second"]<1e6) &
                (data_features["speed"]>0) &
                (data_features["speed"]<60)]

In [6]:
data_features.shape

(12783185, 11)

In [0]:
filename = "/content/drive/My Drive/safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv" 
data_labels = pd.read_csv(filename,header=0)

In [0]:
data_labels = data_labels.drop_duplicates("bookingID", keep="last")

In [0]:
data = pd.merge(data_features, data_labels, how="inner", on="bookingID")

In [0]:
data.sort_values(["bookingID","second"], inplace=True)

In [0]:
singleRide = data[data.bookingID == 0]

plt.figure(figsize=(15,5))
plt.plot(singleRide["second"], singleRide[["acceleration_x","acceleration_y","acceleration_z","Speed"]])
plt.legend(labels=list(singleRide[["acceleration_x","acceleration_y","acceleration_z","Speed"]]), loc='upper left')
plt.title("acceleration")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(singleRide["second"], singleRide[["gyro_x","gyro_y","gyro_z"]])
plt.legend(labels=list(singleRide[["gyro_x","gyro_y","gyro_z"]]), loc='upper left')
plt.title("gyro")
plt.show()

In [0]:
singleRide = data[data.bookingID == 2]

speed = singleRide["Speed"].values
bearing = singleRide["Bearing"].values
time = singleRide["second"].values
map_x = [0]
map_y = [0]
for i in range(len(speed)):
    deltaTime = time[i] - (time[i-1] if i>0 else 0)
    prevSpeed = (speed[i-1] if i>0 else 0)
    distance = 0.5 * (speed[i]+prevSpeed) * deltaTime
    bearing_rad = bearing[i]*2*math.pi/360
    displacementX = math.sin(bearing_rad)*distance
    displacementY = math.cos(bearing_rad)*distance
    new_x = map_x[i] + displacementX
    new_y = map_y[i] + displacementY
    map_x.append(new_x)
    map_y.append(new_y)
plt.figure(figsize=(15,10))
plt.plot(map_x, map_y, marker="o")

for i in range(len(map_x)):
  plt.annotate(i, (map_x[i], map_y[i]))

plt.show()

In [0]:
singleRide.head()

Using sckit-learn

reduce sample size

In [0]:
data_small = data

In [12]:
data['bookingID'].value_counts().value_counts().

405     52
210     50
300     45
285     43
435     42
270     42
240     42
195     41
135     40
479     40
255     38
225     38
315     37
476     37
480     37
165     35
510     35
326     35
330     35
180     35
600     35
120     35
465     34
345     34
495     33
444     33
268     33
150     33
450     33
460     33
        ..
1896     1
55       1
1958     1
1445     1
6        1
1461     1
1493     1
1541     1
1637     1
3700     1
1669     1
1701     1
1797     1
1829     1
1925     1
2037     1
2183     1
1942     1
2583     1
3415     1
1606     1
1622     1
1638     1
1654     1
1670     1
1798     1
1830     1
1846     1
1862     1
40       1
Name: bookingID, Length: 1772, dtype: int64

In [0]:
X = data_small[["bookingID",
      "accuracy",
      "bearing",
      "acceleration_x","acceleration_y","acceleration_z",
      "gyro_x","gyro_y","gyro_z",
      "second",
      "speed"]]
y = data_small[["label"]]

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaling

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler().fit(X_train)

In [0]:
X_train_standardised = scaler.transform(X_train)

In [0]:
X_test_standardised = scaler.transform(X_test)

In [89]:
y_train.shape

(9587388, 1)

trying KERAS

In [0]:
chk = ModelCheckpoint('train_model_all_data.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1)

In [91]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=11,kernel_initializer='uniform',activation='relu'))
model.add(Dense(8,kernel_initializer='uniform',activation='relu'))
model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy','mse'])

history = model.fit(X_train_standardised, y_train.values, validation_data=(X_test_standardised, y_test), epochs=100, batch_size=32, callbacks=[chk], verbose=2)

Train on 9587388 samples, validate on 3195797 samples
Epoch 1/100
 - 430s - loss: 0.5766 - acc: 0.7194 - mean_squared_error: 0.1949 - val_loss: 0.5763 - val_acc: 0.7199 - val_mean_squared_error: 0.1946

Epoch 00001: val_acc improved from -inf to 0.71989, saving model to train_model_all_data.h5
Epoch 2/100
 - 437s - loss: 0.5751 - acc: 0.7205 - mean_squared_error: 0.1943 - val_loss: 0.5739 - val_acc: 0.7207 - val_mean_squared_error: 0.1939

Epoch 00002: val_acc improved from 0.71989 to 0.72074, saving model to train_model_all_data.h5
Epoch 3/100
 - 429s - loss: 0.5749 - acc: 0.7210 - mean_squared_error: 0.1940 - val_loss: 0.5751 - val_acc: 0.7212 - val_mean_squared_error: 0.1938

Epoch 00003: val_acc improved from 0.72074 to 0.72123, saving model to train_model_all_data.h5
Epoch 4/100
 - 430s - loss: 0.5754 - acc: 0.7209 - mean_squared_error: 0.1940 - val_loss: 0.5745 - val_acc: 0.7210 - val_mean_squared_error: 0.1938

Epoch 00004: val_acc did not improve from 0.72123
Epoch 5/100
 - 432

KeyboardInterrupt: ignored

In [0]:
saved_model = load_model('train_model_all_data.h5')

In [100]:
model.metrics_names

['loss', 'acc', 'mean_squared_error']

In [106]:
saved_model.evaluate(X_test_standardised, y_test)

3195797/3195797 [==============================] - 79s 25us/step


[0.5833876934721798, 0.721412530270376, 0.19410793877640456]

saved_model: 72%
model: 

In [0]:
y_pred = saved_model.predict_classes(X_test_standardised)

In [113]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [114]:
print(confusion_matrix(y_test, y_pred))

[[2189624   53823]
 [ 836486  115864]]


In [115]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.98      0.83   2243447
           1       0.68      0.12      0.21    952350

    accuracy                           0.72   3195797
   macro avg       0.70      0.55      0.52   3195797
weighted avg       0.71      0.72      0.64   3195797



with validation set to X_test, y_test
accur 81.34%
no validation 82.78%
68% accuracy

In [95]:
plt.plot(history.history['val_acc'], label='train')
plt.show()

KeyError: ignored

In [96]:
model.evaluate(X_test_st)

{'binary_accuracy': [0.7356666666666667,
  0.7568933333396911,
  0.76708,
  0.7759733333396912,
  0.7807600000063578,
  0.7847333333396912,
  0.7892266666666666,
  0.7948,
  0.7994399999936421,
  0.8009066666666667],
 'loss': [0.5512829708671569,
  0.5200139101092021,
  0.5102407257048289,
  0.4982565962568919,
  0.4857482514222463,
  0.4774596139685313,
  0.470515038207372,
  0.46209401168823244,
  0.4555861202335358,
  0.4509739920425415],
 'mean_squared_error': [0.1847954737552007,
  0.17161358640034993,
  0.16734792442798616,
  0.162271468984286,
  0.15769497606595356,
  0.1546275702969233,
  0.15187023184378942,
  0.14872185785055161,
  0.14619705149014792,
  0.1444358829220136],
 'val_binary_accuracy': [0.75204,
  0.7552,
  0.77168,
  0.7782,
  0.78204,
  0.78504,
  0.79052,
  0.79712,
  0.802,
  0.80376],
 'val_loss': [0.5282378631877899,
  0.5187132813167572,
  0.5099598221302033,
  0.493474546289444,
  0.4851880891036987,
  0.47481143047332763,
  0.46633033124923706,
  0.45704

In [0]:
model.save("model.h5")

In [32]:
model.evaluate(X_test_standardised, y_test.values)

25000/25000 [==============================] - 1s 21us/step


[0.25319444396972657, 0.88248, 0.08124124313116074]

100 epochs: acc = 87.4%
10 epochs: acc = 83.5%

100,000 lines of data
10 epochs: acc = 73%
100 epochs: acc 77%



In [0]:
model.metric_names

AttributeError: ignored

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [0]:
param_grid = {
    'alpha': [1e-4],
              #, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
    'loss': ['hinge'],
}

In [0]:
sgd = SGDClassifier(verbose=100,max_iter=100)
sgd_cv= GridSearchCV(sgd,param_grid,cv=5,verbose=100)
sgd_cv.fit(X_train,y_train)

In [0]:
sgd_cv.best_score_

In [0]:
sgd_cv.score(X_test, y_test)

In [0]:
sgd_log = SGDClassifier(verbose=100, loss='log')
sgd_log.fit(X_train,y_train)

saving our model down

In [0]:
import pickle
pickle.dumps(sgd_cv)